In [1]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import backtrader.feeds as btfeeds
from backtrader import CommInfoBase
import pyfolio, random
import time, math, sys,os, six, concurrent.futures
import pyfolio as pf
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table
import itertools
from multiprocessing import Pool
from datetime import timedelta
import datetime
import plotly.express as px


/Users/marcluiz/opt/anaconda3/lib/python3.8/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


Cerebro Class

In [1]:
class LaneCerebro:
    """
    setting custom dynamic subclass for strategy deployment
    """

    def __init__(self, order_df, signal_df, data_df):
        self.order_df = order_df
        self.signal_df = signal_df
        self.data_df = data_df
        self.broker_cash = 100000
        self.start = None
        self.end = None
        

    def analyze(self):
        # start timer
        self.start = time.time()
        print('------------------------')
        print('Analyzing Strategy....')
        cerebro = bt.Cerebro()

        # 1. Add the datafeed to Cerebro
        print('1/5 Adding datafeed....')
        total_bars = len(self.data_df)
        self.data_df = PandasData.import_data(self.data_df)
        test = self.data_df.merge(self.signal_df, left_index=True, right_index=True)
        cerebro.adddata(PandasData(dataname=test))

        # 2. Set broker parameters
        print('2/5 Setting broker parameters...')
        cerebro.broker.setcash(self.broker_cash)
        cerebro.broker.setcommission(self.order_df.Commission[0], commtype=CommInfoBase.COMM_FIXED)
        cerebro.broker.set_slippage_fixed(fixed=self.order_df['Slippage'][0], slip_open=True, slip_match=False, slip_limit=True)
        
        # 3. Adding Strategy
        print('3/5 Adding Strategy...')
        cerebro.addstrategy(LaneStrategy, 
                            order_types=self.order_df['Order Type'],
                            order_dir=self.order_df['Direction'],
                           profit_targets=self.order_df['Profit Target'],
                           stop_losses=self.order_df['Stop Loss'],
                           limit_prices=self.order_df['Limit Price'],
                           cancel_mins=self.order_df['Cancel Order'],
                           adjust_vol=self.order_df['Adjust Volatility'],
                           expiry_mins=self.order_df['Max Hold'],
                           force_eod=self.order_df['Force EOD'],
                           quantities=self.order_df['Quantity'])
        
         # 4. Set analyzers
        print('4/5 Adding analyzers...')
        cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='TradeAnalyzer')
        cerebro.addanalyzer(btanalyzers.PyFolio)
        cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='Sharpe', timeframe = bt.TimeFrame.Minutes, riskfreerate=0)
        cerebro.addanalyzer(btanalyzers.DrawDown, _name='Drawdown')
        
        # Print out the starting conditions
        print('------------------------')
        print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
        print('------------------------')

        # 5. Run over everything
        strats = cerebro.run()
        strat = strats[0]

        # Performance Metrics
        performance_data = strat.analyzers.TradeAnalyzer.get_analysis()
        pyfolio = strat.analyzers.getbyname('pyfolio')
        returns, positions, transactions, gross_leverage = pyfolio.get_pf_items()
        
        # printing results
        print('------------------------')
        print('Final Portfolio Value: %.4f' % cerebro.broker.getvalue())
        self.end = time.time()
        print(f'Total Time Elapsed : {round((self.end - self.start), 2)}')
        print('-----------------')

        # plotting performance
        cumret = (1 + returns).cumprod() - 1
        return_df = pd.DataFrame({'ret':returns,'cumret':cumret})
        
        fig = px.line(return_df, x=return_df.index, y='cumret')
        fig.show()
        
        pf.create_returns_tear_sheet(returns,
                    positions=positions,
                    transactions=transactions, return_fig = True)

Custom Strategy Class

In [3]:
class LaneStrategy(bt.Strategy):
    """
    setting parameter input
    """
    params = (
        ('order_types', []),
        ('order_dir', []),
        ('profit_targets', []),
        ('stop_losses', []),
        ('limit_prices', []),
        ('cancel_mins',[]),
        ('adjust_vol',[]),
        ('expiry_mins',[]),
        ('force_eod',[]),
        ('quantities',[]),
    )
    
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        self.signal1 = self.datas[0].signal1
        self.signal2 = self.datas[0].signal2
        self.signal3 = self.datas[0].signal3
        self.signal4 = self.datas[0].signal4
        self.vol = self.datas[0].vol
        self.base_sizer = 1
        
        self.orefs = {}
        
        self.signal_dict = {'signal1':{'position':False, 'type':self.p.order_types[0], 'dir':self.p.order_dir[0],'pt':self.p.profit_targets[0],'sl':self.p.stop_losses[0],'lp':self.p.limit_prices[0],'cancel':self.p.cancel_mins[0], 'adjust_vol':self.p.adjust_vol[0], 'expiry':self.p.expiry_mins[0], 'force_eod':self.p.force_eod[0], 'quantity':self.p.quantities[0]},
                           'signal2':{'position':False, 'type':self.p.order_types[1], 'dir':self.p.order_dir[1],'pt':self.p.profit_targets[1],'sl':self.p.stop_losses[1],'lp':self.p.limit_prices[1],'cancel':self.p.cancel_mins[1], 'adjust_vol':self.p.adjust_vol[1], 'expiry':self.p.expiry_mins[1], 'force_eod':self.p.force_eod[1], 'quantity':self.p.quantities[1]},
                           'signal3':{'position':False, 'type':self.p.order_types[2], 'dir':self.p.order_dir[2],'pt':self.p.profit_targets[2],'sl':self.p.stop_losses[2],'lp':self.p.limit_prices[2],'cancel':self.p.cancel_mins[2], 'adjust_vol':self.p.adjust_vol[2], 'expiry':self.p.expiry_mins[2], 'force_eod':self.p.force_eod[2], 'quantity':self.p.quantities[2]},
                           'signal4':{'position':False, 'type':self.p.order_types[3], 'dir':self.p.order_dir[3],'pt':self.p.profit_targets[3],'sl':self.p.stop_losses[3],'lp':self.p.limit_prices[3],'cancel':self.p.cancel_mins[3], 'adjust_vol':self.p.adjust_vol[3], 'expiry':self.p.expiry_mins[3], 'force_eod':self.p.force_eod[3], 'quantity':self.p.quantities[3]}}
    
    def next(self):
        if int(self.signal1[0])!=0 and self.signal_dict['signal1']['position']==0:
            self.create_order('signal1')
            
        if int(self.signal2[0])!=0 and self.signal_dict['signal2']['position']==0:
            self.create_order('signal2')
            
        if int(self.signal3[0])!=0 and self.signal_dict['signal3']['position']==0:
            self.create_order('signal3')
            
        if int(self.signal4[0])!=0 and self.signal_dict['signal4']['position']==0:
            self.create_order('signal4')
            
    def create_order(self,signal_id):
        pt = self.signal_dict[signal_id]['pt']
        sl = self.signal_dict[signal_id]['sl']
        force_eod = self.signal_dict[signal_id]['force_eod']
        exec_type = bt.Order.Market
        size = self.signal_dict[signal_id]['quantity']
        
        # ORDER TYPE LOGIC
        if self.signal_dict[signal_id]['type'] == 'MARKET':
            target_price = self.dataopen[0]
            exec_type = bt.Order.Market
            
        elif self.signal_dict[signal_id]['type'] == 'LIMIT':
            if self.signal_dict[signal_id]['dir'] == 'SHORT':
                target_price = self.dataopen[0] + limit_price
            elif self.signal_dict[signal_id]['dir'] == 'LONG':
                target_price = self.dataopen[0] - limit_price
            exec_type=bt.Order.Limit
            
        # ORDER CANCEL LOGIC
        if self.signal_dict[signal_id]['expiry'] >= 0:
            cancel_mins = timedelta(minutes=self.signal_dict[signal_id]['cancel'])
        expiry_mins = timedelta(minutes=self.signal_dict[signal_id]['expiry'])
        expiry_mins_num = expiry_mins.total_seconds()/60
        eod_mins = 100000
        
        if force_eod == True:
            current_time = (self.data.datetime.datetime(0)).time()
            eod_time = timedelta(hours=23, minutes=59)
            current_time_td = timedelta(hours=current_time.hour, minutes=current_time.minute)
            eod_mins = (eod_time - current_time_td).total_seconds()/60
        
        if eod_mins < expiry_mins_num:
            expiry_mins = eod_mins
        
        
        # ORDER SIZE LOGIC
        if self.signal_dict[signal_id]['adjust_vol'] == True:
            size = math.floor(size/self.vol[0])
        if size==0:
            size=1
        
        
        
        # ORDER DIRECTION LOGIC
        if self.signal_dict[signal_id]['dir'] == 'SHORT':
            pt_price = target_price - pt
            sl_price = target_price + sl
            
            # ADJUSTING FOR VOL
            if self.signal_dict[signal_id]['adjust_vol'] == True:
                pt_price = target_price - (pt_price*self.vol[0])
                sl_price = target_price + (sl_price*self.vol[0])
            
            # MAKING ORDER
            os = self.sell_bracket(exectype=exec_type,
                              limitprice=pt_price,
                              price=target_price,
                              stopprice=sl_price, 
                                  stopexec=bt.Order.Stop,
                                  limitexec=bt.Order.Limit, 
                                  valid=cancel_mins, 
                                  stopargs=dict(valid=expiry_mins), 
                                  limitargs=dict(valid=expiry_mins))
            self.orefs[signal_id] = [o.ref for o in os]
            self.signal_dict[signal_id]['position'] = 1
            
        elif self.signal_dict[signal_id]['dir'] == 'LONG':
            pt_price = target_price + pt
            sl_price = target_price - sl
            
            # ADJUSTING FOR VOL
            if self.signal_dict[signal_id]['adjust_vol'] == True:
                pt_price = target_price + (pt_price*self.vol[0])
                sl_price = target_price - (sl_price*self.vol[0])
            
            # MAKING ORDER
            os = self.buy_bracket(exectype=exec_type,
                              limitprice=pt_price,
                              price=target_price,
                              stopprice=sl_price,
                                  stopexec=bt.Order.Stop,
                                  limitexec=bt.Order.Limit, 
                                  valid=cancel_mins, 
                                  stopargs=dict(valid=expiry_mins), 
                                  limitargs=dict(valid=expiry_mins))
            self.orefs[signal_id] = [o.ref for o in os]
            self.signal_dict[signal_id]['position'] = 1
            
            
    def notify_order(self, order):
        # find orders signal
        signal_id = ''
        for signal in self.orefs.keys():
            if order.ref in self.orefs[signal]:
                signal_id = signal
        
        
        if order.status == order.Completed:
            print(f'{self.data.datetime.datetime(0)}: {signal_id} {"Buy" * order.isbuy() or "Sell"} order {order.ref} {order.getstatusname()}')
            self.signal_dict[signal_id]['position'] = 0
            
        elif order.status == order.Canceled and order.ref in self.orefs:
            self.orefs[signal_id].remove(order.ref)
            self.signal_dict[signal_id]['position'] = 0
            
        elif order.status == order.Expired and order.ref in self.orefs:
            if self.signal_dict[signal_id]['dir'] == 'LONG' and order.issell():
                print(f'{self.data.datetime.datetime(0)} ORDER EXPIRED, EXITING POSITIONS FOR {order.ref}')
                self.sell(size=self.main_order_size)
                self.orefs.remove(order.ref)

            elif self.signal_dict[signal_id]['dir'] == 'SHORT' and order.isbuy():
                print(f'{self.data.datetime.datetime(0)} ORDER EXPIRED, EXITING POSITIONS FOR {order.ref}')
                self.buy(size=self.main_order_size)
                self.orefs.remove(order.ref)
                
        else:
            print(f'{self.data.datetime.datetime(0)}: {signal_id} {"Buy" * order.isbuy() or "Sell"} order {order.ref} {order.getstatusname()}')
            
            
        if not order.alive() and order.ref in self.orefs[signal_id]:
            self.orefs[signal_id].remove(order.ref)
        

Pandas Data Conversion

In [4]:
class PandasData(btfeeds.PandasData):
    """
    PandasData subclasses
    """
    lines = ('up', 'down','vol','signal1','signal2','signal3','signal4')
    params = (
        ('nocase', True),

        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the column in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('datetime', None),

        # Possible values below:
        #  None : column not present
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('open', 0),
        ('high', 1),
        ('low',  2),
        ('close', 3),
        ('up', 4),
        ('down', 5),
        ('vol', 6),
        ('signal1',7),
        ('signal2',8),
        ('signal3',9),
        ('signal4',10),
    )

    datafields = btfeeds.PandasData.datafields + (
        [
            'up', 'down','vol','signal1','signal2', 'signal3', 'signal4'
            
        ]
    )
    
    def import_data(df):
        if len(df.columns) == 6:
            df['vol']=1
        return df

In [5]:
# class LaneStrategy(bt.Strategy):
#     """
#     setting parameter input
#     """
#     params = (
#         ('order_types', []),
#         ('order_dir', []),
#         ('profit_targets', []),
#         ('stop_losses', []),
#         ('limit_prices', []),
#         ('cancel_mins',[]),
#         ('adjust_vol',[]),
#         ('expiry_mins',[]),
#         ('force_eod',[])
#     )
    
#     def __init__(self):
#         self.dataclose = self.datas[0].close
#         self.dataopen = self.datas[0].open
#         self.signal1 = self.datas[0].signal1
#         self.signal2 = self.datas[0].signal2
#         self.signal3 = self.datas[0].signal3
#         self.signal4 = self.datas[0].signal4
#         self.vol = self.datas[0].vol
#         self.base_sizer = 1
        
#         self.orefs = []
#         self.current_orders = []
        
#         self.signal_dict = {'signal1':{'position':False, 'type':self.p.order_types[0], 'dir':self.p.order_dir[0],'pt':self.p.profit_targets[0],'sl':self.p.stop_losses[0],'lp':self.p.limit_prices[0],'cancel':self.p.cancel_mins[0], 'adjust_vol':self.p.adjust_vol[0], 'expiry':self.p.expiry_mins[0], 'force_eod':self.p.force_eod[0]},
#                            'signal2':{'position':False, 'type':self.p.order_types[1], 'dir':self.p.order_dir[1],'pt':self.p.profit_targets[1],'sl':self.p.stop_losses[1],'lp':self.p.limit_prices[1],'cancel':self.p.cancel_mins[1], 'adjust_vol':self.p.adjust_vol[1], 'expiry':self.p.expiry_mins[1], 'force_eod':self.p.force_eod[1]},
#                            'signal3':{'position':False, 'type':self.p.order_types[2], 'dir':self.p.order_dir[2],'pt':self.p.profit_targets[2],'sl':self.p.stop_losses[2],'lp':self.p.limit_prices[2],'cancel':self.p.cancel_mins[2], 'adjust_vol':self.p.adjust_vol[2], 'expiry':self.p.expiry_mins[2], 'force_eod':self.p.force_eod[2]},
#                            'signal4':{'position':False, 'type':self.p.order_types[3], 'dir':self.p.order_dir[3],'pt':self.p.profit_targets[3],'sl':self.p.stop_losses[3],'lp':self.p.limit_prices[3],'cancel':self.p.cancel_mins[3], 'adjust_vol':self.p.adjust_vol[3], 'expiry':self.p.expiry_mins[3], 'force_eod':self.p.force_eod[3]}}
    
#     def next(self):
#         if self.signal1!=0 and self.signal_dict['signal1']['position']==0:
#             self.create_order('signal1')
            
#         if self.signal2!=0 and self.signal_dict['signal2']['position']==0:
#             self.create_order('signal2')
            
#         if self.signal3!=0 and self.signal_dict['signal3']['position']==0:
#             self.create_order('signal3')
            
#         if self.signal4!=0 and self.signal_dict['signal4']['position']==0:
#             self.create_order('signal4')
            
#     def create_order(self,signal_id):
#         pt = self.signal_dict[signal_id]['pt']
#         sl = self.signal_dict[signal_id]['sl']
#         force_eod = self.signal_dict[signal_id]['force_eod']
#         exec_type = bt.Order.Market
        
        
#         # ORDER TYPE LOGIC
#         if self.signal_dict[signal_id]['type'] == 'MARKET':
#             target_price = self.dataopen[0]
#             exec_type = bt.Order.Market
            
#         elif self.signal_dict[signal_id]['type'] == 'LIMIT':
#             if self.signal_dict[signal_id]['dir'] == 'SHORT':
#                 target_price = self.dataopen[0] + limit_price
#             elif self.signal_dict[signal_id]['dir'] == 'LONG':
#                 target_price = self.dataopen[0] - limit_price
#             exec_type=bt.Order.Limit
            
            
#         # ORDER CANCEL LOGIC
#         if self.signal_dict[signal_id]['expiry'] >= 0:
#             cancel_mins = timedelta(minutes=self.signal_dict[signal_id]['cancel'])
#         expiry_mins = timedelta(minutes=self.signal_dict[signal_id]['expiry'])
#         expiry_mins_num = expiry_mins.total_seconds()/60
#         eod_mins = 100000
        
#         if force_eod == True:
#             current_time = (self.data.datetime.datetime(0)).time()
#             eod_time = timedelta(hours=23, minutes=59)
#             current_time_td = timedelta(hours=current_time.hour, minutes=current_time.minute)
#             eod_mins = (eod_time - current_time_td).total_seconds()/60
        
#         if eod_mins < expiry_mins_num:
#             expiry_mins = eod_mins
        
        
        
#         # ORDER DIRECTION LOGIC
#         if self.signal_dict[signal_id]['dir'] == 'SHORT':
#             pt_price = target_price - pt
#             sl_price = target_price + sl
            
#             # ADJUSTING FOR VOL
#             if self.signal_dict[signal_id]['adjust_vol'] == True:
#                 pt_price = target_price - (pt_price*self.vol[0])
#                 sl_price = target_price + (sl_price*self.vol[0])
            
#             # MAKING ORDER
#             os = self.sell_bracket(exectype=exec_type,
#                               limitprice=pt_price,
#                               price=target_price,
#                               stopprice=sl_price,
#                                   stopexec=bt.Order.Stop,
#                                   limitexec=bt.Order.Limit, 
#                                   valid=cancel_mins, 
#                                   stopargs=dict(valid=expiry_mins), 
#                                   limitargs=dict(valid=expiry_mins))
#             self.orefs = [o.ref for o in os]
#             self.current_orders = [o for o in os]
#             print(f'Entering position for {signal_id.upper()}, Order Reference: {self.orefs}')
#             self.signal_dict[signal_id]['position'] = 1
            
#         elif self.signal_dict[signal_id]['dir'] == 'LONG':
#             pt_price = target_price + pt
#             sl_price = target_price - sl
            
#             # ADJUSTING FOR VOL
#             if self.signal_dict[signal_id]['adjust_vol'] == True:
#                 pt_price = target_price + (pt_price*self.vol[0])
#                 sl_price = target_price - (sl_price*self.vol[0])
            
#             # MAKING ORDER
#             os = self.buy_bracket(exectype=exec_type,
#                               limitprice=pt_price,
#                               price=target_price,
#                               stopprice=sl_price,
#                                   stopexec=bt.Order.Stop,
#                                   limitexec=bt.Order.Limit, 
#                                   valid=cancel_mins, 
#                                   stopargs=dict(valid=expiry_mins), 
#                                   limitargs=dict(valid=expiry_mins))
#             self.orefs = [o.ref for o in os]
#             self.current_orders = [o for o in os]
#             print(f'Entering position for {signal_id.upper()}, Order Reference: {self.orefs}')
#             self.signal_dict[signal_id]['position'] = 1
            
            
#     def notify_order(self, order):
#         if order.status == order.Completed:
#             print(f'{self.data.datetime.datetime(0)}: {"Buy" * order.isbuy() or "Sell"} order {order.ref} {order.getstatusname()}')
#             print(f'Cash Left: {round(self.broker.get_value(),2)}')
        
#         elif order.status == order.Canceled and order.ref in self.orefs:
#             self.orefs.remove(order.ref)
            
#         elif order.status == order.Expired and order.ref in self.orefs:
#             if self.signal_dict[signal_id]['dir'] == 'LONG' and order.issell():
#                 print(f'{self.data.datetime.datetime(0)} ORDER EXPIRED, EXITING POSITIONS FOR {order.ref}')
#                 self.sell(size=self.main_order_size)
#                 self.orefs.remove(order.ref)

#             elif self.signal_dict[signal_id]['dir'] == 'SHORT' and order.isbuy():
#                 print(f'{self.data.datetime.datetime(0)} ORDER EXPIRED, EXITING POSITIONS FOR {order.ref}')
#                 self.buy(size=self.main_order_size)
#                 self.orefs.remove(order.ref)
                
#         else:
#             print(f'{self.data.datetime.datetime(0)}: {"Buy" * order.isbuy() or "Sell"} order {order.ref} {order.getstatusname()}')
            
            
#         if not order.alive() and order.ref in self.orefs:
#             self.orefs.remove(order.ref)
        